<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Batched-.match-operation" data-toc-modified-id="Batched-.match-operation-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Batched <code>.match</code> operation</a></span></li><li><span><a href="#More-examples" data-toc-modified-id="More-examples-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>More examples</a></span><ul class="toc-item"><li><span><a href="#Computing-matches-Euclidean-similarity" data-toc-modified-id="Computing-matches-Euclidean-similarity-0.2.1"><span class="toc-item-num">0.2.1&nbsp;&nbsp;</span>Computing matches Euclidean similarity</a></span></li><li><span><a href="#Computing-matches-Euclidean-distance" data-toc-modified-id="Computing-matches-Euclidean-distance-0.2.2"><span class="toc-item-num">0.2.2&nbsp;&nbsp;</span>Computing matches Euclidean distance</a></span></li><li><span><a href="#Computing-matches-Cosine-similarity" data-toc-modified-id="Computing-matches-Cosine-similarity-0.2.3"><span class="toc-item-num">0.2.3&nbsp;&nbsp;</span>Computing matches Cosine similarity</a></span></li></ul></li><li><span><a href="#test-distances-with-scipy" data-toc-modified-id="test-distances-with-scipy-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>test distances with scipy</a></span></li><li><span><a href="#Document-Memmap" data-toc-modified-id="Document-Memmap-0.4"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>Document Memmap</a></span><ul class="toc-item"><li><span><a href="#Test-distances-using-memmaparrays" data-toc-modified-id="Test-distances-using-memmaparrays-0.4.1"><span class="toc-item-num">0.4.1&nbsp;&nbsp;</span>Test distances using memmaparrays</a></span></li></ul></li></ul></li><li><span><a href="#Batching-distance-computation" data-toc-modified-id="Batching-distance-computation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Batching distance computation</a></span><ul class="toc-item"><li><span><a href="#Expressing-the-problem" data-toc-modified-id="Expressing-the-problem-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Expressing the problem</a></span></li><li><span><a href="#Building-the-function" data-toc-modified-id="Building-the-function-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Building the function</a></span></li><li><span><a href="#Adapting-batch-funtion-to-memmap-arrays" data-toc-modified-id="Adapting-batch-funtion-to-memmap-arrays-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Adapting batch funtion to memmap arrays</a></span></li><li><span><a href="#Benchmark-selection-of-best-values-from-a-distance-matrix" data-toc-modified-id="Benchmark-selection-of-best-values-from-a-distance-matrix-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Benchmark selection of best values from a distance matrix</a></span></li></ul></li></ul></div>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from jina import Document, DocumentArray
import numpy as np
from jina.types.arrays.memmap import DocumentArrayMemmap

d1 = Document(embedding=np.array([0,0,0,0,1]))
d2 = Document(embedding=np.array([1,0,0,0,0]))
d3 = Document(embedding=np.array([1,1,1,1,0]))
d4 = Document(embedding=np.array([1,2,2,1,0]))

d1_m = Document(embedding=np.array([0,0.1,0,0,0]))
d2_m = Document(embedding=np.array([1,0.1,0,0,0]))
d3_m = Document(embedding=np.array([1,1.2,1,1,0]))
d4_m = Document(embedding=np.array([1,2.2,2,1,0]))
d5_m = Document(embedding=np.array([4,5.2,2,1,0]))

da_1 = DocumentArray([d1, d2, d3, d4])
da_2 = DocumentArray([d1_m, d2_m, d3_m, d4_m, d5_m])


da_1.match(da_2, metric='euclidean',  limit=3)
query = da_1[2]
print(f'query emb = {query.embedding}')
for m in query.matches:
    print('match emb =', m.embedding, 'score =', m.scores['euclidean'].value)

query emb = [1 1 1 1 0]
match emb = [1.  1.2 1.  1.  0. ] score = 0.20000000298023224
match emb = [1.  2.2 2.  1.  0. ] score = 1.5620499849319458
match emb = [1.  0.1 0.  0.  0. ] score = 1.6763054132461548


## ERROR

The following code is not working because

```
proto = self[0].proto.embedding.dense
```

assumes the embedding is dense

In [3]:
import jina
jina.__version__

'2.2.6'

In [4]:
from jina import Document, DocumentArray
import scipy.sparse as sp

d1 = Document(embedding=sp.csr_matrix([[0,0,0,0,1]]))
d2 = Document(embedding=sp.csr_matrix([[1,0,0,0,0]]))
d3 = Document(embedding=sp.csr_matrix([[1,1,1,1,0]]))
d4 = Document(embedding=sp.csr_matrix([[1,2,2,1,0]]))

d1_m = Document(embedding=sp.csr_matrix([[0,0.1,0,0,0]]))
d2_m = Document(embedding=sp.csr_matrix([[1,0.1,0,0,0]]))
d3_m = Document(embedding=sp.csr_matrix([[1,1.2,1,1,0]]))
d4_m = Document(embedding=sp.csr_matrix([[1,2.2,2,1,0]]))
d5_m = Document(embedding=sp.csr_matrix([[4,5.2,2,1,0]]))

da_1 = DocumentArray([d1, d2, d3, d4])
da_2 = DocumentArray([d1_m, d2_m, d3_m, d4_m, d5_m])



In [5]:
da_1.match(da_2, metric='euclidean', limit=3)
query = da_1[2]
print(f'query emb = {query.embedding}')
for m in query.matches:
    print('match emb =', m.embedding, 'score =', m.scores['euclidean'].value)

TypeError: data type '' not understood

In [6]:
da_1[0].proto.embedding

sparse {
  indices {
    buffer: "\000\000\000\000\004\000\000\000"
    shape: 1
    shape: 2
    dtype: "<i4"
  }
  values {
    buffer: "\001\000\000\000\000\000\000\000"
    shape: 1
    dtype: "<i8"
  }
  shape: 1
  shape: 5
}

The `.embeddings` property is not working correctly for sparse arrays.
It should return an sparse array

In [7]:
import scipy.sparse as sp
da_1[0].get_attributes('embedding')

<1x5 sparse matrix of type '<class 'numpy.int64'>'
	with 1 stored elements in COOrdinate format>

In [8]:
da_1[0:3].get_attributes('embedding')

[<1x5 sparse matrix of type '<class 'numpy.int64'>'
 	with 1 stored elements in COOrdinate format>,
 <1x5 sparse matrix of type '<class 'numpy.int64'>'
 	with 1 stored elements in COOrdinate format>,
 <1x5 sparse matrix of type '<class 'numpy.int64'>'
 	with 4 stored elements in COOrdinate format>]

In [9]:
%timeit sp.vstack(da_1[0:3].get_attributes('embedding'))

478 µs ± 26.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### Big stacking of sparse arrays

In [10]:
from jina import Document, DocumentArray
import scipy.sparse as sp

d1 = Document(embedding=sp.csr_matrix([[0,0,0,0,1]]))
for n in [1000, 10_000, 100_000]:
    print(f'\nresult for n={n}')
    da_1 = DocumentArray([d1]*n)
    %time sp.vstack(da_1.get_attributes('embedding'))


result for n=1000
CPU times: user 123 ms, sys: 2.87 ms, total: 126 ms
Wall time: 127 ms

result for n=10000
CPU times: user 1.15 s, sys: 15.2 ms, total: 1.16 s
Wall time: 1.17 s

result for n=100000
CPU times: user 11 s, sys: 118 ms, total: 11.2 s
Wall time: 11.2 s


In [11]:
da_1[1].proto.embedding.sparse

indices {
  buffer: "\000\000\000\000\004\000\000\000"
  shape: 1
  shape: 2
  dtype: "<i4"
}
values {
  buffer: "\001\000\000\000\000\000\000\000"
  shape: 1
  dtype: "<i8"
}
shape: 1
shape: 5

In [12]:
da_1[2].proto.embedding.sparse

indices {
  buffer: "\000\000\000\000\004\000\000\000"
  shape: 1
  shape: 2
  dtype: "<i4"
}
values {
  buffer: "\001\000\000\000\000\000\000\000"
  shape: 1
  dtype: "<i8"
}
shape: 1
shape: 5

## speedup sparse join at byte level

We can create an sparse array from a collection of buffers without any need to create an sparse array for each of the elements

In [14]:

d1 = Document(embedding=sp.csr_matrix([[0,0,0,0,1]]))
d2 = Document(embedding=sp.csr_matrix([[1,0,0,0,0]]))
d3 = Document(embedding=sp.csr_matrix([[1,1,1,1,0]]))
d4 = Document(embedding=sp.csr_matrix([[1,2,2,1,0]]))

da_1 = DocumentArray([d1, d2, d3, d4])

We should be able to retrieve the following dense vector from the proto buffers

In [25]:
da_1[0].embedding.todense()

matrix([[0, 0, 0, 0, 1]])

Note that the vector is stored with indices and values.

We should get that index (0,4) contains our value.

In [31]:
proto_indices_buffer = da_1[0].proto.embedding.sparse.indices.buffer
proto_indices_buffer

b'\x00\x00\x00\x00\x04\x00\x00\x00'

Note though that if we interpret the bits as int64 we don´t get this

In [32]:
np.frombuffer(proto_indices_buffer, dtype=np.int64)

array([17179869184])

We need to interpret them as int32 to get this information

In [28]:
np.frombuffer(proto_indices_buffer, dtype=np.int32)

array([0, 4], dtype=int32)

- **Why is this happening ? ** Why indices are needed to be interpreted as int32 ?

Note that this information is stored in the dtype of the class `DenseNdArrayProto`.


In [39]:
type(da_1[0].proto.embedding.sparse.indices)

jina_pb2.DenseNdArrayProto

In [35]:
proto_indices_buffer

b'\x00\x00\x00\x00\x04\x00\x00\x00'

In [48]:
indices_proto = da_1[0].proto.embedding.sparse.indices
print(indices_proto)
np.frombuffer(proto_indices_buffer, dtype=indices_proto.dtype)

buffer: "\000\000\000\000\004\000\000\000"
shape: 1
shape: 2
dtype: "<i4"



array([0, 4], dtype=int32)

In [22]:
da_1[2].embedding.todense()

matrix([[1, 1, 1, 1, 0]])

In [17]:
da_1[2].embedding

<1x5 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in COOrdinate format>

In [18]:
np.frombuffer(da_1[0].proto.embedding.sparse.values.buffer, 
              da_1[0].proto.embedding.sparse.values.dtype)

array([1])

In [38]:
dtype = da_1[0].proto.embedding.sparse.values.dtype
dtype

'<i8'

In [25]:
# Note that the buffer contains 8 bytes (16 hex numbers) which is an int 64 number
print(da_1[0].proto.embedding.sparse.values.buffer)
print(da_1[0].embedding.todense())

b'\x01\x00\x00\x00\x00\x00\x00\x00'
[[0 0 0 0 1]]


In [42]:
np.frombuffer(da_1[0].proto.embedding.sparse.indices.buffer, 
              dtype=np.uint32)

array([0, 4], dtype=uint32)

In [43]:
np.frombuffer(da_1[0].proto.embedding.sparse.indices.buffer, 
              dtype=dtype)

array([17179869184])

In [67]:
da_1[0].embedding.row, da_1[0].embedding.col

(array([0], dtype=int32), array([4], dtype=int32))

The following example is more interesting

In [79]:
da_1[2].proto.embedding.sparse

indices {
  buffer: "\000\000\000\000\000\000\000\000\000\000\000\000\001\000\000\000\000\000\000\000\002\000\000\000\000\000\000\000\003\000\000\000"
  shape: 4
  shape: 2
  dtype: "<i4"
}
values {
  buffer: "\001\000\000\000\000\000\000\000\001\000\000\000\000\000\000\000\001\000\000\000\000\000\000\000\001\000\000\000\000\000\000\000"
  shape: 4
  dtype: "<i8"
}
shape: 1
shape: 5

In [109]:
print(da_1[2].embedding.row)
print(da_1[2].embedding.col)
print(da_1[2].embedding.data)

[0 0 0 0]
[0 1 2 3]
[1 1 1 1]


In [123]:
values = np.frombuffer(da_1[2].proto.embedding.sparse.values.buffer, np.int64)
values

array([1, 1, 1, 1])

In [124]:
# BUG? WHAT IS THIS
np.frombuffer(da_1[2].proto.embedding.sparse.indices.buffer, 
              dtype=np.int64)

array([          0,  4294967296,  8589934592, 12884901888])

In [162]:
# the coordinates are stored consecutively here
coordinates = np.frombuffer(da_1[2].proto.embedding.sparse.indices.buffer, 
              dtype=np.int32)
coordinates

array([0, 0, 0, 1, 0, 2, 0, 3], dtype=int32)

In [150]:
sp.coo_matrix(([1,1,1,1],
              (coordinates.reshape(2,4)[0,:],
              coordinates.reshape(2,4)[1,:])))

<2x4 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in COOrdinate format>

In [161]:
r = da_1[2].embedding.row
c = da_1[2].embedding.col
d = da_1[2].embedding.data
sp.csr_matrix((d, (r,c))).todense()

matrix([[1, 1, 1, 1]])

## Batched `.match` operation

In [ ]:
from jina import Document, DocumentArray
import numpy as np

d1 = Document(embedding=np.array([0,0,0,0,1]))
d2 = Document(embedding=np.array([1,0,0,0,0]))
d3 = Document(embedding=np.array([1,1,1,1,0]))
d4 = Document(embedding=np.array([1,2,2,1,0]))

da_1 = DocumentArray([d1, d2, d3, d4])

np.random.seed(1234)
x_mat = np.random.random((1000,5))
da_2 = DocumentArray([Document(embedding=x) for x in x_mat])

In [ ]:
da_1.match(da_2, metric='euclidean',  limit=3)
query = da_1[2]
print(f'query emb = {query.embedding}')
for m in query.matches:
    print('match emb =', np.round(m.embedding,3), 'score =', round(m.scores['euclidean'].value,3))

In [ ]:
da_1.match(da_2, metric='euclidean',  limit=3, batch_size=4)
query = da_1[2]
print(f'query emb = {query.embedding}')
for m in query.matches:
    print('match emb =', np.round(m.embedding,3), 'score =', round(m.scores['euclidean'].value,3))

In [ ]:
!rm -rf dam2
dam_2 = DocumentArrayMemmap('./dam2')
dam_2.extend([Document(embedding=x) for x in x_mat])

da_1.match(dam_2, metric='euclidean',  limit=3, batch_size=100)
query = da_1[2]
print(f'query emb = {query.embedding}')
for m in query.matches:
    print('match emb =', np.round(m.embedding,3), 'score =', round(m.scores['euclidean'].value,3))

In [ ]:
from jina.math.helper import  update_rows_x_mat_best

In [ ]:
x_mat_best = np.array([[1,5], 
                       [3,5]])

x_inds_best = np.array([[0,1], 
                        [0,1]])

x_mat = np.array([[0,1,8,9],
                  [9,8,2,1]])

x_inds = np.array([[4,5,6,7],
                   [4,5,6,7]])

In [ ]:
x_mat_best_new,x_inds_best_new =  update_rows_x_mat_best(x_mat_best, x_inds_best, x_mat, x_inds, 2)

In [ ]:
print(x_mat_best_new)
print(x_inds_best_new)

## Memmap batched match

## More examples

In [ ]:
from jina import Document, DocumentArray
import numpy as np
from jina.math.distance import cosine, sqeuclidean, cdist

In [ ]:
n_features = 100

def get_document_arrays():
    d1 = Document(embedding=np.array([0,0,0,0,0]))
    d2 = Document(embedding=np.array([1,0,0,0,0]))
    d3 = Document(embedding=np.array([1,1,1,1,0]))
    d4 = Document(embedding=np.array([1,2,2,1,0]))

    d1_m = Document(embedding=np.array([0,0.1,0,0,0]))
    d2_m = Document(embedding=np.array([1,0.1,0,0,0]))
    d3_m = Document(embedding=np.array([1,1.2,1,1,0]))
    d4_m = Document(embedding=np.array([1,2.2,2,1,0]))
    d5_m = Document(embedding=np.array([4,5.2,2,1,0]))

    D  = DocumentArray([d1,d2,d3,d4])
    D2 = DocumentArray([d1_m, d2_m, d3_m, d4_m, d5_m])
    return D, D2

import scipy.sparse as sp

def get_document_arrays_sparse():
    d1 = Document(embedding=sp.csr_matrix([0,0,0,0,0]))
    d2 = Document(embedding=sp.csr_matrix([1,0,0,0,0]))
    d3 = Document(embedding=sp.csr_matrix([1,1,1,1,0]))
    d4 = Document(embedding=sp.csr_matrix([1,2,2,1,0]))

    d1_m = Document(embedding=sp.csr_matrix([0,0.1,0,0,0]))
    d2_m = Document(embedding=sp.csr_matrix([1,0.1,0,0,0]))
    d3_m = Document(embedding=sp.csr_matrix([1,1.2,1,1,0]))
    d4_m = Document(embedding=sp.csr_matrix([1,2.2,2,1,0]))
    d5_m = Document(embedding=sp.csr_matrix([4,5.2,2,1,0]))

    D  = DocumentArray([d1,d2,d3,d4])
    D2 = DocumentArray([d1_m, d2_m, d3_m, d4_m, d5_m])
    return D, D2

In [ ]:
D, D2 = get_document_arrays()

In [ ]:
E = np.stack(D.get_attributes('embedding'))
E.argsort(axis=1)

#### _compute_dist Euclidean distance

In [ ]:
X = np.stack(D.get_attributes('embedding'))
Y = np.stack(D2.get_attributes('embedding'))

dists = cdist(X, Y, 'euclidean')
dists

### Computing matches Euclidean similarity

By default `D.match` uses a similarity metric.

This means matched results are ordered from highest to lowest

In [ ]:
D, D2 = get_document_arrays()
D.match(D2, metric='euclidean', limit=3)

In [ ]:
print(f'query={D[2].embedding}')
for m in D[2].matches:
    print(m.embedding, 'score =', m.scores['euclidean'].value)

In [ ]:
print(f'query={D[1].embedding}')
for m in D[1].matches:
    print(m.embedding, 'score =', m.scores['euclidean'].value)

In [ ]:
from jina.enums import EmbeddingClsType

D, D2 = get_document_arrays_sparse()
D.match(D2, 
        metric='euclidean',
        limit=3,
        is_sparse=True)
        #embedding_cls_type = EmbeddingClsType.SCIPY_CSR)

In [ ]:
print(f'query={D[1].embedding.todense()}')
for m in D[1].matches:
    print(m.embedding.todense(), 'score =', m.scores['euclidean'].value)

In [ ]:
print(f'query={D[2].embedding.todense()}')
for m in D[2].matches:
    print(m.embedding.todense(), 'score =', m.scores['euclidean'].value)

### Computing matches Euclidean distance

If we want to use a distance we need to specify **`is_distance=True`**.

If we do so we should get results ordered from lowest to highest

In [ ]:
D, D2 = get_document_arrays()
D.match(D2, metric='euclidean', limit=3, is_distance=True)

In [ ]:
print(f'query={D[2].embedding}')
for m in D[2].matches:
    print(m.embedding, 'distance =', m.scores['euclidean'].value)

### Computing matches Cosine similarity

In [ ]:
D, D2 = get_document_arrays()
D.match(D2, metric='cosine', limit=3)

In [ ]:
print(f'query={D[2].embedding}')
for m in D[2].matches:
    print(m.embedding, 'dist=', m.scores['cosine'].value)

In [ ]:
scores = [m.scores['cosine'].value for m in D[2].matches]
scores ==sorted(scores)[::-1]

If we want cosine distance we need to specify `is_distance=True`

In [ ]:
D, D2 = get_document_arrays()
D.match(D2, metric='cosine', limit=3, is_distance=True)

In [ ]:
print(f'query={D[2].embedding}')
for m in D[2].matches:
    print(m.embedding, 'dist=', m.scores['cosine'].value)

In [ ]:
distances = [m.scores['cosine'].value for m in D[2].matches]
distances ==sorted(distances)

In [ ]:
[m.scores['cosine'].value for d in D for m in d.matches]

## test distances with scipy

In [ ]:

from scipy.spatial.distance import cdist

In [ ]:
#X = np.random.random((10,5))
#Y = np.random.random((15,5))

X = np.array([[1,1,1],[4,5,6],[0,1,2]])
Y = np.array([[1,1,2],[2,3,4]])
XYdistances = cdist(X,Y,metric='euclidean')
XY_new = np.sqrt(euclidean_distance_squared(X,Y))
np.testing.assert_almost_equal(XYdistances,XY_new)

In [ ]:
XYdistances = cdist(X,Y,metric='cosine')
XY_new = cosine_distance(X,Y)
np.testing.assert_almost_equal(XYdistances, XY_new)

In [ ]:
XY_new

In [ ]:
XYdistances

In [ ]:
X

In [ ]:
np.linalg.norm(X, axis=1)

In [ ]:
np.linalg.norm(Y, axis=1)

In [ ]:
np.outer(np.linalg.norm(X, axis=1), np.linalg.norm(Y, axis=1))  

In [ ]:
np.outer(np.array([1,2,3]), np.array([3,4]))

In [ ]:
2.44948974 * 1.73205081

## Document Memmap

In [ ]:
from jina.types.arrays.memmap import DocumentArrayMemmap

##### test_match_docarray_docarraymemmap

In [ ]:
metric = 'euclidean'
D1, D2 = get_document_arrays()
D1.match(D2, metric=metric, limit=3, is_distance=True)
values_docarray = [m.scores[metric].value for d in D1 for m in d.matches]

!rm -rf ./my-memmap
D1, D2 = get_document_arrays()
D2memmap = DocumentArrayMemmap('./my-memmap')
D2memmap.extend(D2)
D1.match(D2memmap, metric=metric, limit=3, is_distance=True)
values_docarraymemmap = [m.scores[metric].value for d in D1 for m in d.matches]

In [ ]:
values_docarraymemmap == values_docarraymemmap

##### test_docarraymemmap_match_docarray

In [ ]:
D1, D2 = get_document_arrays()
D1.match(D2, metric='euclidean', limit=3, is_distance=True)
values_docarray = [m.scores['euclidean'].value for d in D1 for m in d.matches]

!rm -rf ./my-memmap

from jina import Document
D1, D2 = get_document_arrays()
D1memmap = DocumentArrayMemmap('./my-memmap')
D1memmap.extend(D1)
D1memmap.match(D2, metric='euclidean', limit=3, is_distance=True)

print(f'query={dam[2].embedding}')
for m in D1memmap[2].matches:
    print(m.embedding, 'distance =', m.scores['euclidean'].value)
    
values_docarraymemmap = [m.scores['euclidean'].value for d in D1memmap for m in d.matches]

In [ ]:
values_docarray == values_docarraymemmap

##### test_docarraymemmap_match_docarraymemmap

In [ ]:
metric = 'euclidean'
is_distance = 'False'
D1, D2 = get_document_arrays()
D1.match(D2, metric=metric, limit=3, is_distance=True)
values_docarray = [m.scores[metric].value for d in D1 for m in d.matches]

!rm -rf ./my-memmap1
!rm -rf ./my-memmap2
D1, D2 = get_document_arrays()
D1memmap = DocumentArrayMemmap('./my-memmap1')
D1memmap.extend(D1)
D1memmap.reload()
D2memmap = DocumentArrayMemmap('./my-memmap2')
D2memmap.extend(D2)
D2memmap.reload()
D2memmap.reload()
D1memmap.match(D2memmap, metric=metric, limit=3, is_distance=is_distance)

#values_docarraymemmap = [m.scores[metric].value for d in D1memmap for m in d.matches]


In [ ]:
values_docarraymemmap = [m.scores[metric].value for d in D1memmap for m in d.matches]
values_docarraymemmap

In [ ]:
D1memmap[0].matches

In [ ]:
values_docarray

### Test distances using memmaparrays

In [ ]:
euclidean_distance_squared(D1memmap, D2memmap)

In [ ]:
D1memmap.get_attributes('embedding')

# Batching distance computation

## Expressing the problem


Let `x_mat` and `y_mat` be two matrices with `n_x` and `n_y` rows respectively.

The ouput of our code is two matrices:

- `dists`: is a np.ndarray of shape `(n_x, k)` containing at row `i` the closest `k` distances from `x_mat[i,:]` and the `best` elements from `y_mat`. This matrix does not store which elements from `Y` are selected. This information is in the following matrix `inds`. 


- `inds`: is a np.ndarray of shape `(n_x, k)` containing at row `i` the indices of the `k` closest selected items from `y_mat`


We want to compute the pariwise distances between `x_mat` and `y_mat` assuming that rows in `x_mat` can be stored in memory but not all rows from `y_mat`.  Therefore we will load from `y_mat` in batches and compute the distances with each batch.


In [ ]:
import jina.math 
from jina.math.distance import sqeuclidean
from jina.math.helper import top_k

## Building the function

In [ ]:
n_features = 10
np.random.seed(1234)
X = np.random.random((10, n_features))
Y = np.random.random((1000,n_features))

print(X[0]) 
print(np.argpartition([4,0,2,6,9,1],kth=3))
print(sqeuclidean(X,Y).shape)

In [ ]:
dists, inds = top_k(sqeuclidean(X,Y), 10)
print(inds.shape, dists.shape)
print(dists[0])
print(inds[0])

In [ ]:
def batch_generator(y_mat, n_batch):
    for i in range(0,len(y_mat), n_batch):
        yield y_mat[i:i+n_batch], i 

In [ ]:
[(a[0].shape,a[1]) for a in batch_generator(Y, n_batch=100)]

In [ ]:
def batch_top_k(x_mat, y_mat, batch_size, k):

    n_x = x_mat.shape[0]

    y_batch_generator = batch_generator(y_mat, batch_size)
    top_dists = np.inf * np.ones((n_x, k))
    top_inds  = np.zeros((n_x, k), dtype=int)
    
    for ybatch, ybatch_start_pos in y_batch_generator:
        distances = sqeuclidean(x_mat, ybatch)
        dists, inds = top_k(distances, k)
        inds = ybatch_start_pos + inds
        top_dists, top_inds = get_min_dists(dists, inds, top_dists, top_inds, k)
        
    # sort final the final `top_dists` and `top_inds` per row 
    perm = np.argsort(top_dists, axis=1)
    top_dists = np.take_along_axis(top_dists, perm, axis=1)
    top_inds = np.take_along_axis(top_inds, perm, axis=1)
    
    return top_dists, top_inds

In [ ]:
def get_min_dists(dists, inds, top_dists, top_inds, k):
    all_dists = np.hstack((dists, top_dists))
    all_inds = np.hstack((inds, top_inds))
    best_inds = np.argpartition(all_dists, kth=k, axis=1)
    d_mat = np.take_along_axis(all_dists, best_inds , axis=1)
    i_mat = np.take_along_axis(all_inds, best_inds , axis=1)
    return d_mat[:,:k], i_mat[:,:k]

def get_min_dists2(dists, inds, top_dists, top_inds, k):
    all_dists = np.hstack((dists, top_dists))
    all_inds = np.hstack((inds, top_inds))
    best_inds = np.argpartition(all_dists, kth=k, axis=1)[:, 0:k]
    d_mat = np.array([all_dists[i,best_inds[i]] for i in range(len(all_dists))])
    i_mat = np.array([all_inds[i,best_inds[i]] for i in range(len(all_dists))])
    return d_mat, i_mat

In [ ]:
n_features = 20
k = 5
batch_size = 100

np.random.seed(1234)
X = np.random.random((10, n_features))
Y = np.random.random((1000, n_features))

In [ ]:
batch_top_k(X, Y, batch_size, k=k)

In [ ]:
top_k(sqeuclidean(X,Y), k=k)

## Adapting batch funtion to memmap arrays 

In [ ]:
def get_min_dists(dists, inds, top_dists, top_inds, k):
    all_dists = np.hstack((dists, top_dists))
    all_inds = np.hstack((inds, top_inds))
    best_inds = np.argpartition(all_dists, kth=k, axis=1)
    d_mat = np.take_along_axis(all_dists, best_inds , axis=1)
    i_mat = np.take_along_axis(all_inds, best_inds , axis=1)
    return d_mat[:,:k], i_mat[:,:k]

def batch_top_k(x_mat, y_mat, batch_size, k):

    n_x = x_mat.shape[0]

    y_batch_generator = batch_generator(y_mat, batch_size)
    top_dists = np.inf * np.ones((n_x, top_k_value))
    top_inds  = np.zeros((n_x, k), dtype=int)
    
    for ybatch, ybatch_start_pos in y_batch_generator:
        distances = sqeuclidean(x_mat, ybatch)
        dists, inds = top_k(distances, k)
        inds = ybatch_start_pos + inds
        top_dists, top_inds = get_min_dists(dists, inds, top_dists, top_inds, k)
        
    # sort final the final `top_dists` and `top_inds` per row 
    perm = np.argsort(top_dists, axis=1)
    top_dists = np.take_along_axis(top_dists, perm, axis=1)
    top_inds = np.take_along_axis(top_inds, perm, axis=1)
    
    return top_dists, top_inds

In [ ]:
from jina.types.arrays.memmap import DocumentArrayMemmap
from jina import Document

n_features = 20
np.random.seed(1234)
X = np.random.random((10, n_features))
Y = np.random.random((1000, n_features))

y_dam = DocumentArrayMemmap('./my-memmap')
y_dam.extend([Document(embedding=y) for y in Y ])
y_dam[0:5]

In [ ]:
n_features = 20

np.random.seed(1234)
X = np.random.random((10, n_features))
Y = np.random.random((1000, n_features))

y_da = DocumentArray([Document(embedding=y) for y in Y ])

In [ ]:
y_da[0:5]

In [ ]:
aux = np.memmap(y_dam._body)

In [ ]:
len(aux)

## Benchmark selection of best values from a distance matrix

In [ ]:
aux = np.vstack([y_dam[i].embedding for i in range(1000)])

In [ ]:
all_dists = np.hstack((dists, top_dists))
all_inds = np.hstack((inds, top_inds))
best_inds = np.argpartition(all_dists, kth=top_k_value, axis=1)[:,:top_k_value]

In [ ]:
print(all_dists)
print(best_inds)
print(np.take_along_axis(all_dists, best_inds , axis=1))

One way to do it is with a comprenhensive list

In [ ]:
%%timeit
d_mat = np.array([all_dists[i,best_inds[i][:top_k_value]] for i in range(len(all_dists))])

Another way to do it is with `np.take_along_axis`

In [ ]:
%%timeit
np.take_along_axis(all_dists, best_inds , axis=1)

Both methods return the same

In [ ]:
np.array([all_dists[i,best_inds[i][:top_k_value]] for i in range(len(all_dists))])

In [ ]:
np.take_along_axis(all_dists, best_inds , axis=1)

In [ ]:
def f(a=1,b=2,c=None):
    if c:
        return a+b+c
    else:
        return a+b

In [ ]:
f(1,2)

In [ ]:
f(1,2,**{'d':24})

In [ ]:
def g(a=1,b=2):
    return a+b